In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd '/content/drive/MyDrive/ClassAug Work'

/content/drive/.shortcut-targets-by-id/12PFZZk88jXqRuPPPvsCgycke0gd7aQBI/ClassAug Work


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution()
import tensorflow_hub as hub
# import sentencepiece as spm
import pickle
import numpy as np
import pandas as pd
import re
import tensorflow_datasets as tfds

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
tf.executing_eagerly()

True

In [5]:
# train_data = fetch_20newsgroups(subset='train')
# test_data = fetch_20newsgroups(subset='test')
# df = pd.DataFrame(train_data.data + test_data.data, columns=['text'])
# labels = np.hstack((train_data.target, test_data.target))

# train_data = list(tfds.as_numpy(tfds.load('trec', split='train', shuffle_files=True)))
# test_data = list(tfds.as_numpy(tfds.load('trec', split='test', shuffle_files=True)))
# df = pd.DataFrame(train_data.data + test_data.data, columns=['text'])
# labels = np.hstack((train_data.target, test_data.target))

# df = pd.read_csv('SSTDataset.txt', sep='\t', header=None)
{i: x for i, x in enumerate(pd.read_csv('./embeddings/twitter us airline sentiment.csv')['airline_sentiment'].unique())}

# df = pd.read_csv('./embeddings/sms_spam_dataset.csv', encoding='latin')

{0: 'neutral', 1: 'positive', 2: 'negative'}

In [ ]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [ ]:
df = df[['v1', 'v2']]
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
# train_data = list(map(lambda x: x['text'].decode(), train_data))
# test_data = list(map(lambda x: x['text'].decode(), test_data))

In [ ]:
# df = pd.DataFrame(train_data + test_data, columns=['text'])

In [ ]:
# label_mapping = {}
# for i, c in enumerate(train_data.target_names):
#   label_mapping[c] = i
label_mapping = {}
for i, c in enumerate(df['v1'].unique()):
  label_mapping[c] = i

In [ ]:
label_mapping

{'ham': 0, 'spam': 1}

In [ ]:
DATASET_NAME = 'sms_spam'

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
tf.disable_eager_execution()
with tf.Session(config=config) as sess:
    module = hub.Module("universal-sentence-encoder-lite_2")

input_placeholder = tf.sparse_placeholder(tf.int64, shape=[None, None])
encodings = module(inputs=dict(values=input_placeholder.values, indices=input_placeholder.indices, dense_shape=input_placeholder.dense_shape))
## Load SentencePiece model from the TF-Hub Module
#### The SentencePiece model is conveniently stored inside the module's assets. It has to be loaded in order to initialize the processor.

with tf.Session(config=config) as sess:
    spm_path = sess.run(module(signature="spm_path"))

sp = spm.SentencePieceProcessor()
sp.Load(spm_path)
print("SentencePiece model loaded at {}.".format(spm_path))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


SentencePiece model loaded at b'universal-sentence-encoder-lite_2/assets/universal_encoder_8k_spm.model'.


In [ ]:
def process_to_IDs_in_sparse_format(sp, sentences):
    # An utility method that processes sentences with the sentence piece processor
    # 'sp' and returns the results in tf.SparseTensor-similar format:
    # (values, indices, dense_shape)
    ids = [sp.EncodeAsIds(x) for x in sentences]
    max_len = max(len(x) for x in ids)
    dense_shape=(len(ids), max_len)
    values=[item for sublist in ids for item in sublist]
    indices=[[row,col] for row in range(len(ids)) for col in range(len(ids[row]))]
    return values, indices, dense_shape

In [ ]:
def generate_embeddings(data):
    """
    param data: str
    n_chars: int
    """
    values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, data)
    
    with tf.Session(config=config) as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embeddings = session.run(
        encodings,
        feed_dict={input_placeholder.values: values,
                    input_placeholder.indices: indices,
                    input_placeholder.dense_shape: dense_shape})
        
    return embeddings

In [ ]:
def preprocess_text(text):
    n_chars=np.random.randint(1,3001)
    clean_regex='[^A-Za-z0-9 ]'
    text = re.sub(clean_regex, ' ', text)
    text = re.sub(' +', ' ', text)
    return text[:n_chars]

In [ ]:
df['v2'] = df['v2'].apply(preprocess_text)
docs = df['v2'].to_list()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
testdoc = None
with open('./embeddings/testdoc.txt', 'r') as f:
  testdoc = str(f.read())
testdoc = preprocess_text(testdoc)
print(testdoc)
testdoc = generate_embeddings([testdoc])
with open('./embeddings/testdoc.pkl', 'wb') as f:
  pickle.dump(testdoc, f)

 Sam Rowe SENIOR LECTURER IN RUMINANT MEDICINE UNIVERSITY OF SYDNEY 410 Werombi Rd Brownlow Hill 2570 NSW Australia samuel rowe sydney edu au sam rowe Education Doctorate of Philosophy St Paul MN USA UNIVERSITY OF MINNESOTA Aug 2017 Mar 2020 Major Veterinary medicine Minor Epidemiology Committee Sandra Godden Scott McDougall Richard Maclehose Erin Royster Luciano Caixeta Master of Veterinary Medicine Palmerston North New Zealand MASSEY UNIVERSITY Sep 2012 Jan 2017 Major Production animal medicine Advisor Richard Laven Bachelor of Veterinary Science Townsville QLD Australia JAMES COOK UNIVERSITY Jan 2006 Dec 2010 Skills Epidemiology Observational studies Clinical trials Bias analysis Hierarchical data Test validation Biostatistics Regression Mixed models GEE Structural Equation Modelling Bayesian Latent Class models Programming R Stata SAS Latex Markdown Other software Git DairyCOMP 305 PCDART DairyWin CPM Dairy AminoCow Standard Office suites Clinical skills Farm consultancy Advanced c

In [ ]:
embs = []
labels = []

for i in range(0, len(docs), 1000):
  embs.extend(generate_embeddings(docs[i:i+1000]))
  labels.extend(list(map(lambda x: label_mapping[x], df['v1'][i:i+1000])))
  print(i)

0
1000
2000
3000
4000
5000


In [ ]:
# all_embs = np.empty(shape=(0, 512))
# for i in range(len(embs)):
#     all_embs = np.vstack((all_embs, embs[i]))
embs = np.array(embs)
labels = np.array(labels)

In [ ]:
with open('./embeddings/%s_embeddings.pkl' % DATASET_NAME, 'wb') as f:
  pickle.dump({
      "embs": embs,
      "labels": labels,
      # "label_mapping": label_mapping
  }, f)

In [ ]:
embs.shape

(5572, 512)